In [1]:
import pandas as pd
import pickle

In [2]:
# Unpickle the DataFrame
with open('my_dataframe.pkl', 'rb') as f:
    gdf = pickle.load(f)

# Display the DataFrame
print(gdf)

     H  V  sY  sX                                           geometry
0   16  4   0   0  POLYGON ((-150584.000 2564804.000, -150584.000...
0   16  4   1   0  POLYGON ((-150584.000 2579804.000, -150584.000...
0   16  4   2   0  POLYGON ((-150584.000 2594804.000, -150584.000...
0   16  4   3   0  POLYGON ((-150584.000 2609804.000, -150584.000...
0   16  4   4   0  POLYGON ((-150584.000 2624804.000, -150584.000...
..  .. ..  ..  ..                                                ...
0   16  4   5   9  POLYGON ((-15584.000 2639804.000, -15584.000 2...
0   16  4   6   9  POLYGON ((-15584.000 2654804.000, -15584.000 2...
0   16  4   7   9  POLYGON ((-15584.000 2669804.000, -15584.000 2...
0   16  4   8   9  POLYGON ((-15584.000 2684804.000, -15584.000 2...
0   16  4   9   9  POLYGON ((-15584.000 2699804.000, -15584.000 2...

[100 rows x 5 columns]


In [3]:
import geopandas as gd
fargo_gdf = gd.GeoDataFrame.from_file('fargo.geojson')

In [4]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [5]:
gdf.crs

<Projected CRS: PROJCS["Albers_Conic_Equal_Area",GEOGCS["WGS 84",D ...>
Name: Albers_Conic_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
fargo_gdf

,geometry
0,"POLYGON ((-96.98795 46.96361, -96.98795 46.791..."


In [7]:
fargo_albers_gdf=fargo_gdf.to_crs(gdf.crs)

In [8]:
aoi_gdf = gd.overlay(gdf, fargo_albers_gdf, how='intersection')

In [9]:
aoi_gdf.crs

<Projected CRS: PROJCS["Albers_Conic_Equal_Area",GEOGCS["WGS 84",D ...>
Name: Albers_Conic_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
aoi_gdf

,H,V,sY,sX,geometry
0,16,4,5,5,"POLYGON ((-75584.000 2654804.000, -75584.000 2..."
1,16,4,6,5,"POLYGON ((-75584.000 2654804.000, -75623.662 2..."
2,16,4,5,6,"POLYGON ((-75584.000 2654804.000, -60584.000 2..."
3,16,4,6,6,"POLYGON ((-60584.000 2654804.000, -75584.000 2..."
4,16,4,5,7,"POLYGON ((-60584.000 2654804.000, -51645.447 2..."
5,16,4,6,7,"POLYGON ((-60584.000 2654804.000, -60584.000 2..."


In [11]:
import folium
def simple_map(my_geo):
    center=my_geo.geometry.bounds['miny'].values[0], my_geo.geometry.bounds['minx'].values[0]
    sw = center
    ne = my_geo.geometry.bounds['maxy'].values[0], my_geo.geometry.bounds['maxx'].values[0]
    map3 = folium.Map(location=center, zoom_start=12,center=center)
    map3.fit_bounds([sw,ne])
    my_geo_json = my_geo.to_json()
    style_function= lambda x:{'color':'red'}
    folium.GeoJson(my_geo_json,style_function).add_to(map3)
    #map3.add_child(folium.features.LatLngPopup())

    return(map3)

In [12]:
my_geo_json = fargo_gdf.to_json()

In [13]:
m = simple_map(fargo_gdf)

In [14]:
m

In [15]:
def add_df_map(map3, my_gdf, color='yellow'):
    print(my_gdf)
    plot_hv = my_gdf.to_crs('EPSG:4326')
    my_geo_json = plot_hv.to_json()
    style_function= lambda x:{'color':color}
    folium.GeoJson(my_geo_json,style_function).add_to(map3)

In [16]:
for i, r in aoi_gdf.iterrows():
    print(r)
    print(type(r['geometry']))
    my_df_list=[]
    rec = {
        'H': r['H'],
        'V': r['V'],
        'sY': r['sY'],
        'sX': r['sX'],
        'geometry': r['geometry']
    }
    my_df_list.append(rec)
    
    gdf1 = gd.GeoDataFrame(my_df_list, crs = aoi_gdf.crs)
    # geo_gdf=gd.GeoDataFrame(geometry=r['geometry'], crs = aoi_gdf.crs)
    add_df_map(m, gdf1)

H                                                          16
V                                                           4
sY                                                          5
sX                                                          5
geometry    POLYGON ((-75584 2654804, -75584 2645034.42395...
Name: 0, dtype: object
<class 'shapely.geometry.polygon.Polygon'>
    H  V  sY  sX                                           geometry
0  16  4   5   5  POLYGON ((-75584.000 2654804.000, -75584.000 2...
H                                                          16
V                                                           4
sY                                                          6
sX                                                          5
geometry    POLYGON ((-75584 2654804, -75623.66175802126 2...
Name: 1, dtype: object
<class 'shapely.geometry.polygon.Polygon'>
    H  V  sY  sX                                           geometry
0  16  4   6   5  POLYGON ((-75584.000 26548

In [17]:
m